In [ ]:
import fiona
import numpy as np
import pandas as pd
from wit_tooling import *
from wit_tooling.database.io import DIO
from datetime import datetime
dio = DIO.get()

In [ ]:
# the necessaties for bokeh plot 
from ipywidgets import interact
from IPython.display import display
from bokeh.io import output_notebook, push_notebook, show
from bokeh.layouts import layout
from bokeh.models import YearsTicker,DatetimeTickFormatter, Legend, Button, CategoricalColorMapper, ColumnDataSource,HoverTool, Label, SingleIntervalTicker, Slider
from bokeh.palettes import plasma, Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
output_notebook()

In [ ]:
# there are 26 catchments in the database you can have all
catchment_list = dict()
for i in range(26): 
    catchment_name= dio.get_name_by_id('catchments', i+1)[0][0] 
    if catchment_name == '': 
        continue
    catchment_list[i+1] = catchment_name
print(catchment_list)

In [ ]:
# pick the polygons included by one/some catchment(s), don't pick too many though
# or else...you can have a try to see...
rows = dio.get_polys_by_catchment_id(1, 5000) 
poly_list = list(np.array(rows)[:,0])
print(poly_list)

In [ ]:
# start from here you can do a lot of things with polygons
# e.g. plot the stack plot as we did
# pick a poly_id from poly_list above
poly_id = poly_list[0]
# get the data
poly_data = get_area_by_poly_id(int(poly_id))
# get the name
poly_name = dio.get_name_by_id("polygons", int(poly_id))[0][0]
print(poly_name)
print(poly_data)
# tidy up a bit
poly_area = poly_data.area.unique()[0]
poly_data = poly_data.drop(columns=['area'])

In [ ]:
# find a polygon from your shape file and match one in the database
shapefile = "/g/data/u46/users/ea6141/wlinsight/shapefiles/LTIMRamsarProj_3577.shp"

with fiona.open(shapefile) as allshapes:
    start_f = iter(allshapes)
    while True:
        shape = next(start_f)
        feature_id = int(shape['id'])
        print(feature_id)
        object_id = int(shape['properties']['OBJECTID'])
        if feature_id != 1:
            continue
        # or it could be object id in the properties of the polygon
        # if object_id != XX:
        #     continue
        poly_name, count = query_wit_data(shape)
        break


In [ ]:
# save the csv and the plot
output_path = './'
pd.DataFrame(data=count, columns=['TIME', 'BS', 'NPV', 'PV', 'WET', 'WATER']).to_csv(output_path+str(shape['id'])+'.csv', index=False)
b_image = plot_to_png(count, poly_name)
with open(output_path+str(shape['id'])+'.png', 'wb') as f:
    f.write(b_image.read())


In [ ]:
# now plot
sub_plot = figure(y_range=(0, poly_area), x_range=(poly_data['time'].min(), poly_data['time'].max()), title=poly_name,
        plot_height=200, plot_width=900, sizing_mode='stretch_width')

sub_plot.xaxis.formatter = DatetimeTickFormatter()
sub_plot.xaxis.ticker = YearsTicker(interval=1)
sub_plot.yaxis.axis_label = "Area (hectare)"

pal = [ '#030aa7',
        '#04d9ff',
        '#3f9b0b',
        '#e6daa6',
        '#60460f'
    ]
v_stack = sub_plot.varea_stack(['open water', 'wet', 'green veg', 'dry veg', 'bare soil'], x='time',
                         color=pal, source=poly_data, alpha=0.6)
legend = Legend(items=[
("bare soil", [v_stack[4]]),
("dry veg", [v_stack[3]]),
("green veg", [v_stack[2]]),
("wet", [v_stack[1]]),
("open water", [v_stack[0]]),
], location="top_left")

sub_plot.add_layout(legend, 'left')
show(sub_plot)

In [ ]:
# here you can get all time metrics for one/some polygon(s)
results = get_alltime_metrics(poly_list[1:5])
print(results)

In [ ]:
# here you get yearly metrics for one/some polygon(s)
results = get_wet_year_metrics(poly_list[1:5])
print(results)

In [ ]:
results = get_pv_year_metrics(poly_list[0:500])
print(results)

In [ ]:
results = get_year_metrics_with_type_area(poly_list[0:500])

In [ ]:
results

In [ ]:
results.area = results.area/100 * np.pi

In [ ]:
plot = figure(y_range=(0, 1), 
              x_range=(0, 1), title='DN', plot_height=700, plot_width=900)
plot.xaxis.ticker = SingleIntervalTicker(interval=0.1)
plot.xaxis.axis_label = "Water"
plot.yaxis.ticker = SingleIntervalTicker(interval=0.1)
plot.yaxis.axis_label = "PV"

#label = Label(x=1.1, y=18, text=str(years[0]), text_font_size='70pt', text_color='#eeeeee')
#plot.add_layout(label)
color_map = plasma(len(results.type.unique()))
catchment_list = tuple(results.type.unique())
print(color_map)
color_mapper = factor_cmap('type', palette=color_map, factors=catchment_list)
#color_mapper = CategoricalColorMapper(palette=color_map, factors=catchment_list)
d = plot.circle(
    x='water_max',
    y='pv_max',
    size='area',
    source = results.loc[(results.year==1988) & (results.type.isin(['Palustrine', 'Lacustrine']))],
    fill_color=color_mapper,
    fill_alpha=0.5,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5,
    legend_field='type'
)

plot.legend.location = "top_left"
plot.legend.click_policy="mute"

In [ ]:
show(plot, notebook_handle=True)

In [ ]:
def yearly_plot(year=1987, wtype=['Palustrine', 'Lacustrine']):
    if wtype == ['Palustrine', 'Lacustrine']:
        d.glyph.y = 'wet_max'
        plot.yaxis.axis_label = "Wet"
    else:
        d.glyph.y = 'pv_max'
        plot.yaxis.axis_label = "PV"
    d.data_source.data = results.loc[(results.year==year) & (results.type.isin(wtype))]
    push_notebook()

In [ ]:
interact(yearly_plot, year=(1987, 2019), wtype=[('all', list(results.type.unique())),
                                                ('PL', ['Palustrine', 'Lacustrine']),
                                               ('b', ['Floodplain'])])

In [ ]:
# here starts the piece how we do fancy plot of inundation
def inundation_by_catchment(start_year, end_year):
    source = None
    for i in range(2):
        rows = dio.get_polys_by_catchment_id(i+1, 10000)
        poly_list = list(np.array(rows)[:,0])
        print(len(poly_list))
        if source is None:
            start_time = datetime.now()
            source = get_inundation(poly_list, start_year, end_year, 50, 1000)
            print("end query in", datetime.now()-start_time)
            source['catchment'] = str(i+1)
        else:
            start_time = datetime.now()
            tmp = get_inundation(poly_list, start_year, end_year, 50, 1000)
            print("end query in", datetime.now()-start_time)
            tmp['catchment'] = str(i+1)
            source = pd.concat([source, tmp], ignore_index=True)
    return source

In [ ]:
decades = [(1990, 2000), (2000, 2010), (2010, 2020)]
source = None
for d in decades:
    if source is None:
        source = inundation_by_catchment(d[0], d[1])
        source['decade'] = d[1]
    else:
        tmp = inundation_by_catchment(d[0], d[1])
        tmp['decade'] = d[1]
        source = pd.concat([source, tmp], ignore_index=True)

In [ ]:
source.percent = source.percent * 100
source.area = source.area/100 * np.pi

In [ ]:
source[source.decade==2020]

In [ ]:
plot = figure(y_range=(0, 100), 
              x_range=(0, 12), title='Inundation', plot_height=700, plot_width=900)
plot.xaxis.ticker = SingleIntervalTicker(interval=1)
plot.xaxis.axis_label = "Years"
plot.yaxis.ticker = SingleIntervalTicker(interval=10)
plot.yaxis.axis_label = "Percent"

#label = Label(x=1.1, y=18, text=str(years[0]), text_font_size='70pt', text_color='#eeeeee')
#plot.add_layout(label)
color_map = plasma(len(source.catchment.unique()))
catchment_list = tuple(source.catchment.unique())
print(color_map)
color_mapper = factor_cmap('catchment', palette=color_map, factors=catchment_list)
#color_mapper = CategoricalColorMapper(palette=color_map, factors=catchment_list)
c = plot.circle(
    x='wet_years',
    y='percent',
    size='area',
    source = source[source.decade==2000],
    fill_color=color_mapper,
    fill_alpha=0.5,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5,
    legend_field='catchment'
)

plot.legend.location = "top_left"
plot.legend.click_policy="mute"

In [ ]:
def update_plot(decade=2000):
    c.data_source.data = source[source.decade==decade]
    push_notebook()

In [ ]:
interact(update_plot, decade=[2000, 2010, 2020])

In [ ]:
show(plot, notebook_handle=True)